In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import math
from datetime import timedelta, datetime
from pytimekr import pytimekr

In [2]:
# 한글날, 어린이날, 광복절, 현충일, 석가탄신일, 추석(+-1), 설날(+-1)
holiday = []
for i in range(2018,2022):
    holiday.append(pytimekr.hangul(i).strftime("%Y-%m-%d"))
    holiday.append(pytimekr.children(i).strftime("%Y-%m-%d"))
    holiday.append(pytimekr.independence(i).strftime("%Y-%m-%d"))
    holiday.append(pytimekr.memorial(i).strftime("%Y-%m-%d"))
    holiday.append(pytimekr.buddha(i).strftime("%Y-%m-%d"))
    holiday.append(pytimekr.samiljeol(i).strftime("%Y-%m-%d"))
    
    chuseok = pytimekr.chuseok(i).strftime("%Y-%m-%d")
    chuseok_delta_p = datetime.strptime(chuseok,'%Y-%m-%d') + timedelta(1)
    chuseok_delta_m = datetime.strptime(chuseok,'%Y-%m-%d') + timedelta(-1)
    chuseok_delta_p = chuseok_delta_p.strftime("%Y-%m-%d")
    chuseok_delta_m = chuseok_delta_m.strftime("%Y-%m-%d")
    holiday.append(chuseok)
    holiday.append(chuseok_delta_p)
    holiday.append(chuseok_delta_m)    
    
    lunar_newyear = pytimekr.lunar_newyear(i).strftime("%Y-%m-%d")
    lunar_newyear_delta_p = datetime.strptime(lunar_newyear,'%Y-%m-%d') + timedelta(1)
    lunar_newyear_delta_m = datetime.strptime(lunar_newyear,'%Y-%m-%d') + timedelta(-1)
    lunar_newyear_delta_p = lunar_newyear_delta_p.strftime("%Y-%m-%d")
    lunar_newyear_delta_m = lunar_newyear_delta_m.strftime("%Y-%m-%d")
    holiday.append(lunar_newyear)
    holiday.append(lunar_newyear_delta_p)
    holiday.append(lunar_newyear_delta_m)

In [3]:
Data_1 = pd.read_csv('../data/train_data/01_음식물쓰레기_FOOD_WASTE_210811_update.CSV',encoding='cp949', low_memory = False)
Data_2 = pd.read_csv('../data/train_data/02-1_내국인유동인구_KOREAN.CSV',encoding='cp949')
Data_3 = pd.read_csv('../data/train_data/02-2_장기체류 외국인 유동인구_LONG_TERM_FRGN.CSV',encoding='cp949')
Data_4 = pd.read_csv('../data/train_data/02-3_단기체류 외국인 유동인구_SHORT_TERM_FRGN.CSV',encoding='cp949')
Data_5 = pd.read_csv('../data/train_data/03_거주인구_RESIDENT_POP.CSV',encoding='cp949')
Data_6 = pd.read_csv('../data/train_data/04_음식관련 카드소비_CARD_SPENDING.CSV',encoding='cp949')

In [4]:
kisang = pd.read_csv('../data/processed_data/일자별_읍면동별_종간기상관측.csv', encoding='cp949')

In [5]:
kisang.columns = ['일자', '읍면동', '평균기온', '최저기온', '최고기온', '강수 계속시간',
       '일강수량', '최대 순간 풍속', '최대 풍속', '평균 풍속', '평균 상대습도',
       '평균 증기압', '합계 일조시간', '평균 전운량', '평균 지면온도']

In [6]:
kisang = kisang.fillna(0)

In [22]:
# 1. 카드소비 데이터
Data_6 = Data_6.drop(Data_6[Data_6.emd_nm == '우도면'].index)
Data_6 = Data_6.drop(Data_6[Data_6.emd_nm == '추자면'].index)
Data_6['use_mean'] = Data_6['use_amt']/Data_6['use_cnt']
base = Data_6.groupby(['base_date','emd_nm']).mean().reset_index()
base.columns = ['일자','읍면동','일지출횟수합계','소비액합계','회당지출액합계']
print(base.shape[0])

# 2. 내국인유동인구 데이터
Data_2 = Data_2.drop(Data_2[Data_2.emd_nm == '우도면'].index)
Data_2 = Data_2.drop(Data_2[Data_2.emd_nm == '추자면'].index)
temp = Data_2.groupby(['base_date','emd_nm','sex']).sum().reset_index()
temp = temp.drop(['emd_cd','time','age'], axis=1)
sex = ['남성','여성']
temp_x = temp.groupby(['base_date', 'sex']).mean().reset_index()
temp_x = temp_x.insert(1, 'emd_nm', '알수없음')
temp = pd.concat([temp, temp_x])
for i in sex:
    issex = temp.sex == i
    temp_n = temp[issex]
    temp_n.columns = ['일자','읍면동','성별',f'{i}_거주평균',f'{i}_근무평균',f'{i}_방문평균']
    temp_n = temp_n.drop(['성별'],axis=1)
    base = pd.merge(base, temp_n, how='left')
print(base.shape[0])


# 3. 장기체류 외국인 유동인구
Data_3 = Data_3.drop(Data_3[Data_3.emd_nm == '우도면'].index)
Data_3 = Data_3.drop(Data_3[Data_3.emd_nm == '추자면'].index)
temp = Data_3.groupby(['base_date','emd_nm']).sum().reset_index()
temp = temp.drop(['emd_cd','time'],axis=1)
temp_x = temp.groupby(['base_date']).mean().reset_index()
temp_x = temp_x.insert(1, 'emd_nm', '알수없음')
temp = pd.concat([temp, temp_x])
temp.columns = ['일자', '읍면동', '장외_거주평균','장외_근무평균','장외_방문평균']
base = pd.merge(base, temp, how='left')
print(base.shape[0])


# 4. 단기체류 외국인 유동인구    
Data_4 = Data_4.drop(Data_4[Data_4.emd_nm == '우도면'].index)
Data_4 = Data_4.drop(Data_4[Data_4.emd_nm == '추자면'].index)
temp = Data_4.groupby(['base_date','emd_nm']).sum().reset_index()
temp = temp.drop(['emd_cd','time'],axis=1)
temp_x = temp.groupby(['base_date']).mean().reset_index()
temp_x = temp_x.insert(1, 'emd_nm', '알수없음')
temp = pd.concat([temp, temp_x])
temp.columns = ['일자','읍면동', '단외_방문평균']
base = pd.merge(base, temp, how='left')
print(base.shape[0])


# 5. 거주인구
a_0 = pd.read_csv('../data/processed_data/여성_거주인구.csv', encoding='cp949')
a_1 = pd.read_csv('../data/processed_data/남성_거주인구.csv', encoding='cp949')
a_2 = pd.read_csv('../data/processed_data/총_거주인구.csv', encoding='cp949')
base = pd.merge(base, a_0, how='left')
base = pd.merge(base, a_1, how='left')
base = pd.merge(base, a_2, how='left')
print(base.shape[0])

# 6. 기상데이터 합치기
base = pd.merge(base, kisang, how='left')
print(base.shape[0])


# 7. 코로나 전후, 코로나 발생인원
covid = pd.read_csv('../data/processed_data/코로나.csv', encoding='cp949')
covid = covid.drop([447]) ## 2020년 4월 10일 데이터가 두개 행 존재
covid['일자'] = covid.일자.apply(lambda x: x.split(' ')[0])
base = pd.merge(base, covid, on ='일자', how='left')
base['코로나발생여부'] = base['확진자수'].apply(lambda x: 0 if math.isnan(x) else 1)
print(base.shape[0])


# 8. 일평균 제주도 입도객 수
people_come = pd.read_csv('../data/processed_data/일평균입도객평균.csv', encoding='cp949')
base = pd.merge(base, people_come, how='left')
print(base.shape[0])


# 9. 음식물 쓰레기 데이터
temp = Data_1.groupby(['base_date','emd_nm']).sum().reset_index()
temp.columns = ['일자', '읍면동', '일배출횟수합계', '일배출금액합계', '일별배출량합계']
base = pd.merge(base, temp, how='left')
print(base.shape[0])

# 10. 요일변수, 공휴일여부
base['요일변환_임시'] = pd.to_datetime(base['일자'])
base['요일'] = base['요일변환_임시'].apply(lambda x: x.dayofweek)
base['월'] = base['요일변환_임시'].apply(lambda x: x.month)
base['주말여부'] = base['요일'].apply(lambda x: 1 if x in [5,6] else 0)
base['공휴일여부'] = base['일자'].apply(lambda x: 1 if x in holiday else 0)
base['요일변환_임시'] = pd.to_datetime(base['일자'])
base['주말및공휴일여부_임시'] = base['주말여부'] + base['공휴일여부']
base['주말및공휴일여부'] = base['주말및공휴일여부_임시'].apply(lambda x: 1 if  x>=1 else 0)
base = base.drop(columns = ['요일변환_임시','주말여부', '공휴일여부','주말및공휴일여부_임시'])
print(base.shape[0])

# 11. 성수기/비수기
base['성수기여부'] = base['월'].apply(lambda x: 1 if x in [1,2,7,8] else 0)
print(base.shape[0])

# 12. 카드소비 -> 일별내식소비합계, 일별배달소비합계, 일별외식소비합계
a = Data_6.query("mct_cat_nm in ['농축수산물', '마트/슈퍼마켓', '식품']")
temp = a.groupby(['base_date','emd_nm'])['use_amt'].sum().reset_index()
temp.columns = ['일자', '읍면동', '일별내식소비합계']
base = pd.merge(base, temp, how='left')
a = Data_6.query("mct_cat_nm in ['배달']")
temp = a.groupby(['base_date','emd_nm'])['use_amt'].sum().reset_index()
temp.columns = ['일자', '읍면동', '일별배달소비합계']
base = pd.merge(base, temp, how='left')
a = Data_6.query("mct_cat_nm in ['한식', '패스트푸드', '주점및주류판매', '양식', '간식', '부페','아시아음식']")
temp = a.groupby(['base_date','emd_nm'])['use_amt'].sum().reset_index()
temp.columns = ['일자', '읍면동', '일별외식소비합계']
base = pd.merge(base, temp, how='left')
print(base.shape[0])

# 13. 기타 다른 피쳐(새로운파생변수생성.ipynb)
new_temp = pd.read_csv('../data/processed_data/new_feature.csv', encoding='cp949')
base = pd.merge(base, new_temp, how='left')
print(base.shape[0])


53634
53634
53634
53634
53634
53634
53634
53634
53634
53634
53634
53634
53634


In [64]:
non_0 = ['회당배출금액','일배출횟수합계','일배출금액합계','일별배출량합계','회당배출량']

In [65]:
whole_list = list(base.columns)

In [66]:
yes_0 = list(set(set(whole_list)-set(non_0)))

In [67]:
non_0_df = base[non_0]

In [68]:
yes_0_df = base[yes_0]

In [69]:
yes_0_df = yes_0_df.fillna(0)

In [70]:
real_final_fillna = pd.concat([yes_0_df,non_0_df],axis=1)

In [71]:
## 회귀용 데이터 생성
regression = real_final_fillna.dropna(axis=0).reset_index(drop=True)
regression.to_csv('../data/processed_data/regression.csv', index=False, encoding='cp949')

In [56]:
## 모델링용 데이터 생성
modeling = real_final_fillna.fillna(real_final_fillna.mean())
modeling.to_csv('../data/processed_data/modeling.csv', index=False, encoding='cp949')